In [1]:
import h2o
import h2o.grid

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 16 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_started_from_R_raj_szi088
H2O cluster total nodes:,1
H2O cluster free memory:,3.277 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [12]:
url =  "http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip"
data=h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
train,valid,test=data.split_frame([0.8,0.1],seed=69)

In [14]:
print("%d/%d/%d" %(train.nrows,valid.nrows,test.nrows))

35255/4272/4451


In [16]:
data.col_names

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'IsArrDelayed',
 'IsDepDelayed']

In [17]:
y='IsArrDelayed'
ignoreFields=['ArrDelay',
 'DepDelay','CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay','IsArrDelayed',
 'IsDepDelayed','ActualElapsedTime']

In [33]:
xAll=[i for i in train.names if i not in ignoreFields]
xLikely=['Month',
 'DayOfWeek','UniqueCarrier','Origin',
 'Dest','Distance','Cancelled','Diverted']

In [23]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [24]:
m_def= H2OGeneralizedLinearEstimator(family='binomial',)

In [25]:
m_def.train(xAll,y,train,validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
m_def.logloss(valid=True)

0.6233229932181948

In [29]:
g= h2o.grid.H2OGridSearch(
H2OGeneralizedLinearEstimator(family="binomial",
lambda_search=True
),
hyper_params={
    "alpha":[x * 0.01 for x in range(0,100)],
},
search_criteria={
    "strategy":"RandomDiscrete",
    "max_models":8,
    "max_runtime_secs":30
}
)
g.train(xAll,y,train,validation_frame=valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [30]:
g # higher the value of alpha the better

      alpha                                                    model_ids  \
0    [0.87]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_3   
1    [0.84]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_4   
2    [0.74]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_2   
3    [0.72]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_5   
4    [0.64]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_6   
5     [0.6]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_1   
6    [0.44]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_0   
7    [0.02]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_8_model_7   

              logloss  
0  0.5915200975143313  
1  0.5917814922675966  
2  0.5927150572400006  
3  0.5929156642431362  
4  0.5937835708879243  
5   0.594264129887982  
6  0.5966105801407572  
7  0.6239863095303368  


In [34]:
g2= h2o.grid.H2OGridSearch(
H2OGeneralizedLinearEstimator(family="binomial",
lambda_search=True
),
hyper_params={
    "alpha":[0,0.2,0.4,0.5,0.6,0.8,0.99],
},
search_criteria={
    "strategy":"Cartesian"
}
)
g2.train(xLikely,y,train,validation_frame=valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [35]:
g2 #much worse, very little variation based on alpha

      alpha                                                     model_ids  \
0     [0.4]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_2   
1     [0.5]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_3   
2     [0.2]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_1   
3     [0.6]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_4   
4     [0.0]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_0   
5    [0.99]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_6   
6     [0.8]  Grid_GLM_py_8_sid_905a_model_python_1529795838258_25_model_5   

              logloss  
0  0.6487157727187454  
1  0.6487160884951749  
2  0.6487196240162347  
3  0.6487273991184022  
4  0.6487585913496811  
5  0.6487609977067444  
6  0.6487706105598544  


This is telling us it makes sense to use as many fields you can rather than just sensible one